In [17]:
import os
import mido
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
import numpy as np
from utils.preprocess.quantize_note_timings import quantize_note_timings
from utils.preprocess.normalize_velocities import normalize_velocities
from utils.preprocess.filter_unnecessary_data import filter_unnecessary_data
from utils.train.functions import split_train_validation_data, preprocess_data_for_training
from music21 import *
import musescore

In [3]:
def parse_midi_file(midi_file_path):
    midi_data = mido.MidiFile(midi_file_path)
    return midi_data

In [4]:
def preprocess_midi_data(midi_data):
    midi_data = quantize_note_timings(midi_data)
    midi_data = normalize_velocities(midi_data)
    midi_data = filter_unnecessary_data(midi_data)
    return midi_data

In [5]:
def load_midi_files(file_directory):
    midi_files = []
    for root, dirs, files in os.walk(file_directory):
        for file in files:
            if file.endswith(".mid") or file.endswith(".midi"):
                midi_files.append(os.path.join(root, file))
    return midi_files

In [6]:
def train_lstm_model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes):
    print("Train sequences shape: ", train_sequences.shape)
    print("Train labels shape: ", train_labels.shape)
    print("Validation sequences shape: ", validation_sequences.shape)
    print("Validation labels shape: ", validation_labels.shape)
    print("Input shape: ", input_shape)
    print("Number of classes: ", num_classes)
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),
        tf.keras.layers.LSTM(units=128, return_sequences=True),
        tf.keras.layers.LSTM(units=128),
        tf.keras.layers.Dense(units=num_classes, activation='softmax')
    ])
    print(model.summary())
    print("Model input shape: ", model.input_shape)
    print("Model output shape: ", model.output_shape)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_sequences, train_labels, epochs=20, validation_data=(validation_sequences, validation_labels))
    return model, history

In [7]:
def train_machine_learning_model():
    file_directory = "./adl-piano-midi/Children"
    midi_files = load_midi_files(file_directory)
    print("Loaded {} MIDI files".format(len(midi_files)))
    preprocessed_midi_data = [preprocess_midi_data(parse_midi_file(file)) for file in midi_files]
    print("Preprocessed {} MIDI files".format(len(preprocessed_midi_data)))
    train_data, validation_data = split_train_validation_data(preprocessed_midi_data)
    print("Split {} MIDI files into {} training files and {} validation files".format(len(preprocessed_midi_data), len(train_data), len(validation_data)))
    train_sequences, train_labels = preprocess_data_for_training(train_data)
    print("Created {} training sequences and {} training labels".format(len(train_sequences), len(train_labels)))
    validation_sequences, validation_labels = preprocess_data_for_training(validation_data)
    print("Created {} validation sequences and {} validation labels".format(len(validation_sequences), len(validation_labels)))
    input_shape = train_sequences.shape[1:]
    num_classes = np.max(train_labels) + 1
    model, history = train_lstm_model(train_sequences, train_labels, validation_sequences, validation_labels, input_shape, num_classes)
    return model, history

In [38]:
def filter_midi_data(midi_data, model):
    print("Midi data type: ", type(midi_data))
    # Convert the MIDI data to numerical format suitable for the model
    preprocessed_data = preprocess_midi_data(midi_data)
    print("Preprocessed data type: ", type(preprocessed_data))

    # Perform the prediction using the preprocessed data
    predictions = model.predict(preprocessed_data)
    return predictions

In [9]:
def convert_midi_to_music_representation(midi_data):
    music_rep = converter.parse(midi_data)
    return music_rep

In [10]:
def generate_sheet_music(music_representation):
    sheet_music = music_representation
    return sheet_music

In [11]:
def export_sheet_music(sheet_music, output_format, filename):
    sheet_music.write(output_format, fp=filename)

In [14]:
if __name__ == "__main__":
    root = tk.Tk()
    root.withdraw()
    midi_file_path = filedialog.askopenfilename()
    midi_data = parse_midi_file(midi_file_path)
    preprocess_midi_data(midi_data)
    print("MIDI file loaded and preprocessed")

MIDI file loaded and preprocessed


In [18]:
model = train_machine_learning_model()
print("Model trained")


Loaded 24 MIDI files
Preprocessed 24 MIDI files
Split 24 MIDI files into 19 training files and 5 validation files
Created 41516 training sequences and 41516 training labels
Created 9392 validation sequences and 9392 validation labels
Train sequences shape:  (41516, 32, 2)
Train labels shape:  (41516,)
Validation sequences shape:  (9392, 32, 2)
Validation labels shape:  (9392,)
Input shape:  (32, 2)
Number of classes:  104


2023-05-26 19:13:07.182411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-26 19:13:07.186138: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-26 19:13:07.188440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32, 128)           67072     
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 104)               13416     
                                                                 
Total params: 212,072
Trainable params: 212,072
Non-trainable params: 0
_________________________________________________________________
None
Model input shape:  (None, 32, 2)
Model output shape:  (None, 104)
Epoch 1/20


2023-05-26 19:13:07.534790: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-26 19:13:07.536717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-26 19:13:07.539797: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1297/1298 [============================>.] - ETA: 0s - loss: 3.2837 - accuracy: 0.1140

2023-05-26 19:14:04.641815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-26 19:14:04.646485: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-26 19:14:04.650176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1298/1298 [==============================] - 62s 43ms/step - loss: 3.2835 - accuracy: 0.1140 - val_loss: 3.3634 - val_accuracy: 0.0911
Epoch 2/20
1298/1298 [==============================] - 55s 42ms/step - loss: 2.9079 - accuracy: 0.1681 - val_loss: 3.3847 - val_accuracy: 0.0784
Epoch 3/20
1298/1298 [==============================] - 52s 40ms/step - loss: 2.7242 - accuracy: 0.2059 - val_loss: 3.5452 - val_accuracy: 0.0794
Epoch 4/20
1298/1298 [==============================] - 53s 41ms/step - loss: 2.5905 - accuracy: 0.2348 - val_loss: 3.4972 - val_accuracy: 0.0800
Epoch 5/20
1298/1298 [==============================] - 53s 41ms/step - loss: 2.4735 - accuracy: 0.2609 - val_loss: 3.5898 - val_accuracy: 0.0826
Epoch 6/20
1298/1298 [==============================] - 53s 41ms/step - loss: 2.3713 - accuracy: 0.2850 - val_loss: 3.6238 - val_accuracy: 0.0763
Epoch 7/20
1298/1298 [==============================] - 53s 41ms/step - loss: 2.2753 - accuracy: 0.3087 - val_loss: 3.6428 - val_accura

AttributeError: 'tuple' object has no attribute 'summary'

In [28]:
print("Sequential model show summary", model[0].summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32, 128)           67072     
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 104)               13416     
                                                                 
Total params: 212,072
Trainable params: 212,072
Non-trainable params: 0
_________________________________________________________________
Sequential model show summary None


In [40]:
filter_midi_data(midi_data, model[0])

Midi data type:  <class 'mido.midifiles.midifiles.MidiFile'>
Preprocessed data type:  <class 'mido.midifiles.midifiles.MidiFile'>


ValueError: Failed to find data adapter that can handle input: <class 'mido.midifiles.midifiles.MidiFile'>, <class 'NoneType'>

In [ ]:
music_representation = convert_midi_to_music_representation(midi_file_path)

In [ ]:
sheet_music = generate_sheet_music(music_representation)

In [ ]:
output_file_path_XML = "./resultXML/" + os.path.splitext(os.path.basename(midi_file_path))[0] + ".xml"
output_file_path_PDF = "./resultPDF/" + os.path.splitext(os.path.basename(midi_file_path))[0] + ".pdf"

In [ ]:
export_sheet_music(sheet_music, "musicxml", output_file_path_XML)